In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Data augmentation and normalization for training and validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.ColorJitter(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

In [4]:
# Paths to datasets
data_dir = 'Indian'
image_datasets = {
    x: datasets.ImageFolder(root=f'{data_dir}/{x}', transform=data_transforms[x])
    for x in ['train', 'val', 'test']
}
data_loaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
    for x in ['train', 'val', 'test']
}
class_names = image_datasets['train'].classes
num_classes = len(class_names)

In [5]:
# Load pre-trained ResNet-50 model
resnet50 = models.resnet50(pretrained=True)

c:\Users\amrit\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\amrit\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Freeze all layers initially
for param in resnet50.parameters():
    param.requires_grad = False

# Unfreeze the final layers for fine-tuning
for name, param in resnet50.named_parameters():
    if "layer4" in name:  # Unfreeze the last residual block
        param.requires_grad = True

In [7]:
# Replace the fully connected layer
resnet50.fc = nn.Sequential(
    nn.Linear(resnet50.fc.in_features, 512),  # Add a hidden layer
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes),  # Output layer
    nn.Softmax(dim=1)
)
resnet50 = resnet50.to(device)

In [8]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, resnet50.parameters()), lr=0.0001)

In [9]:
# Learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

c:\Users\amrit\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [ ]:
# Training loop with progress bar
num_epochs = 20
best_acc = 0.0

for epoch in range(num_epochs):
    print(f'Epoch {epoch+1}/{num_epochs}')
    print('-' * 10)
    
    for phase in ['train', 'val']:
        if phase == 'train':
            resnet50.train()
        else:
            resnet50.eval()
        
        running_loss = 0.0
        running_corrects = 0
        
        data_iter = tqdm(data_loaders[phase], desc=f'{phase} Phase', leave=False)
        for inputs, labels in data_iter:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            with torch.set_grad_enabled(phase == 'train'):
                outputs = resnet50(inputs)
                loss = criterion(outputs, labels)
                _, preds = torch.max(outputs, 1)
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(image_datasets[phase])
        epoch_acc = running_corrects.double() / len(image_datasets[phase])
        
        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        if phase == 'val':
            scheduler.step(epoch_loss)
            if epoch_acc > best_acc:
                best_acc = epoch_acc
                torch.save(resnet50.state_dict(), 'best_resnet50_model.pth')

print(f'Best val Acc: {best_acc:.4f}')



Epoch 1/20
----------


train Phase:  75%|███████▌  | 805/1069 [01:27<00:25, 10.18it/s]

In [21]:
# Save the final model
torch.save(resnet50.state_dict(), 'resnet50_fmodel.pth')

In [7]:
from tqdm import tqdm

# Load the saved model
resnet50 = models.resnet50(pretrained=False)  # Use the same architecture
resnet50.fc = nn.Sequential(
    nn.Linear(resnet50.fc.in_features, 512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes),  # Output layer
    nn.Softmax(dim=1)
)
resnet50.load_state_dict(torch.load('resnet50_fmodel.pth'))
resnet50 = resnet50.to(device)
resnet50.eval()

# Evaluate on the test dataset
running_corrects = 0
total_samples = 0

data_iter = tqdm(data_loaders['test'], desc='Testing Phase')
for inputs, labels in data_iter:
    inputs, labels = inputs.to(device), labels.to(device)
    
    with torch.no_grad():
        outputs = resnet50(inputs)
        _, preds = torch.max(outputs, 1)
    
    running_corrects += torch.sum(preds == labels.data)
    total_samples += labels.size(0)

# Compute the accuracy
test_acc = running_corrects.double() / total_samples
print(f'Test Accuracy: {test_acc:.4f}')


c:\Users\amrit\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\amrit\AppData\Local\Temp\ipykernel_20200\442310463.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by 

Test Accuracy: 1.0000
